# 04: Effective Go

These are notes from https://go.dev/doc/effective_go

## Objectives 

TBD

## Introduction

This guide will let you understand Go's properties and idioms. Additionally, it will be explained Go's conventions relative to naming, formatting, program construction, etc.

| NOTE: |
| :---- |
| This [guide](https://go.dev/doc/effective_go) was written in 2009, and hasn't been updated much since. While it continues to be useful, a few of the examples will feel outdated. |

## Formatting

Go includes the *gofmt* program, available as `go fmt` that reads a Go program and emits the source in a standard style of indentation and vertical alignment, retaining and if necessary reformatting comments.

For example:

The *untidied*:
```go
type T struct {
	name string // name of the object
	value int // its value
}
```

Will be transformed by *gofmt* into:

```go
type T struct {
	name  string // name of the object
	value int    // its value
}
```

| NOTE: |
| :---- |
| You must configure VSCode with `gofmt` in the settings. |

Additionally:
+ Tabs are used for indentation, and *gofmt* emits them by default. Spaces should be used only if you must.

    Because of this, it's not that important whether you set your tab size to 2, 4 or 8. I use two because it makes the code more compact horizontally.

+ There is no length limit in Go. If a line feels too long, wrap it and indent with an extra tab.

+ Go needs fewer parentheses than C and Java.

## Commentary

Go supports C-style `/* */` block comments and C++-style `//` line comments, with the latter being the norm.

## Names

Names in Go are extremely important, as they even have semantic effects (e.g., the visibility of a name outside of a package is determined by whether its first character is upper case).

### Package names

When a package name is imported, the package name becomes the accessor for the contents.

That is, after:

```go
import "bytes"
```

you can do `bytes.Buffer`.

By convention, packages are given lowercase, single-word names; there should be no need for underscore or mixedcaps. The package names should be short, concise, and evocative.

Another convention is that the package name is the base name of its source directory. For example, a package in `src/encoding/base64` will be imported as `import "encoding/base64"` and the package name will be `base64` and not `encoding_base64` or similar.

Additionally, think of the end-users: when exposing a function in your package use a concise name like `Reader` instead of `BufReader`, as the users will refer to it as `bufio.Reader`.

Similarly, when creating new instances of a particular type defined in a package, it is recommended to use something like `New()` instead of `NewRing()` if the package only exports a single type (as it should). This is because the end-users will code `ring.New()` to refer to it.

In Go, short names with a helpful doc comment is preferred over a long name:

```go
// discouraged
once.DoOrWaitUntilDone(setup)

// preferred
once.Do(setup) // blocks until setup is completed
```

### Getters

Go doesn't provide automatic support for getters and setters. 

This doesn't mean that you cannot use them, but it's not consider idiomatic, nor necessary to put `Get` into the getter's name.

In Go, it's preferred to use the name of the object (uppercase, exported) instead of `GetField()`, while for the setter it is OK to use `SetField()`

```go
// Person is a struct used in the sample
type Person struct {
	name string // name of the object
	age  int
}

// Name is the Person.name getter
func (p *Person) Name() string {
	return p.name
}

// Age is the Person.age getter
func (p *Person) Age() int {
	return p.age
}

// SetName is the Person.name setter
func (p *Person) SetName(name string) {
	p.name = name
}

// Age is the Person.name setter
func (p *Person) SetAge(age int) {
	p.age = age
}

func main() {
	var p Person

	p.SetName("Jason Isaacs")
	p.SetAge(55)

	fmt.Println(p)

	fmt.Printf("Hello to %v\n", p.Name())
}
```

### Interface names

By convention, one-method interfaces are named by the method name plus an *-er* suffix or similar:

```go
type Reader interface {
  Read(f *File, b []byte) (n int, err error)
}
```

There are a number of such names (`Reader`, `Writer`, `Formatter`, `CloseNotifier`...) and it's productive to honor them and the function names they capture.

There are also canonical signatures and meanings such as `String()`, `Error()`, `Close()`, `Flush()`, etc.

To avoid confusion, don't give your method such names unless it has the same signature and meaning.

Conversely, if your type implements a method with the same meaning as a method on a well-known type, give it the same name and signature (e.g., call your string converter method `String()` and not `ToString()`).

```go
// Person is a struct used in the sample
type Person struct {
	name string // name of the object
	age  int
}

func (p *Person) String() string {
	return fmt.Sprintf("%q (%v)", p.name, p.age)
}

func main() {
	var p *Person = &Person{"Jason Isaacs", 56}

	fmt.Println(p)
}
```

### MixedCaps

The convention in Go is to use MixedCaps or mixedCaps rather than underscores to write multiWord names.

## Semicolons

Go's formal grammar uses semicolons to terminate statements, but unlike C, those semicolons do not appear in the source &mdash; the lexer includes them automatically as it scans the source code.

However, the rules the lexer uses imposes certain conventions, such as having to put the opening brace of a control structure in the same line and not on the next line:

```go
// Correct
if i < f() {
  g()
}

// WRONG!
if i < f()
{
  g()
}
```

| NOTE: |
| :---- |
| The compiler and IDE will warn you of that even with an "unexpected newline" error. |

## Control structures

In Go:
+ There is no `do` or `while` loop, but there is a generalized form of `for` that fits the bill.
+ `switch` is far more flexible than in other programming languages.
+ `if` and `switch` accept an optional initialization statement (like the one you have in `for` loops).
+ `break` and `continue` support an optional label to identify what to break or continue.
+ There is a type `switch`
+ There is a `select` multiway communication multiplexer.


### If

In Go a simple `if` looks like:

```go
if x > 0 {
  return y
}
```

That is, you need to use the braces no matten how simple the if body is.

"If" statement in Go supports an initialization statement:

```go
if err := file.Chmod(0664); err != nil {
  log.Print(err)
  return err
}
```

In Go, it is also considered idiomatic to code if statements that doesn't flow into the next statement &mdash; that is, the body ends in break, continue, goto, or return:

```go
f, err := os.Open(name)
if err != nil {
  return err
}

codeUsing(f)
```

The following code shows another example in which you will find no "else" statements. In it, flow control runs does the page, eliminating error cases as they arise:

```go
f, err := os.Open(name)
if err != nil {
  return err  // Couldn't open the file
}

d, err := f.Stat()  // redeclaration of err? (see below)
if err != nil {
  f.Close()
  return err  // Couldn't stat the file
}

codeUsing(f, d)
```

### Redeclaration and assignment

In the last example, we see that we do:

```go
f, err := os.Open(name)
```

and a few lines later:

```go
d, err := f.Stat()
```

The duplication of `err` is legal: `err` is declared by the first statement, and only re-assigned in the second (that is, the call to `f.Stat()` does not declares a new variable, it just gives it a new value).

In a `:=` declaration, a variable `v` may appear even if it has already been declared, provided that:

+ this declaration is in the same scope as the existing declaration of `v`. If `v` is already declared in an outer scope, the declaration will create a new variable,

+ the corresponding value in the initialization is assignable to `v`, and

+ there is at least one other variable that is created by the declaration.

### For

The Go for loop is similar to C's, but its capabilities are enhanced to support for and while.

```go
// full for form
for init; condition; post { ... }

// similar to C's while
for condition { ... }

// while true
for { ... }
```

If you're looping over an array, slice, string, or map, or reading from a channel you can use the *range* clause to have a sort of foreach:

```go
for key, value := range oldMap {
  newMap[key] = value
}

// You can omit the second value if not needed
for key := range m {
  if key.expiered() {
    delete(m, key)
  }
}

// You can use the blank identifier if you only need the second value
for _, value := range arr {
  sum += value
}
```

For strings, the *range* clause breaks out individual Unicode code points by parsing the UTF-8. Erroneus encodings consume one byte and product the replacement rune U+FFFD:

```go
// x80 is an illegal UTF-8 encoding
for pos, char := range "日本\x80語" {
  fmt.Printf("character %#U starts at byte pos %d", char, pos)
}
```

Go has no comma operator and `++` and `--` are statements, not expressions. As a result, if you want to run multiple variables in a single for loop you need to use parallel assignment like:

```go
for i, j := 0, len(a)-1; i < j; i, j = i+1, j-1 {
  ...
}
```

### Switch

In Go's switch, the expressions need not to be constants or even integers, the cases are evaluated top to bottom until a match is found, and if the switch has no expression it switches on `true`.

It's there possible, and considered idiomatic, to write if-elseif-elseif chain as switch:

```go
func unhex(c byte) byte {

  // switch { means execute the firs branch which is true
  switch {
  case '0' <= c && c <= '9':
    return c - '0'
  case 'a' <= c && c <= 'f':
    return c - 'a' + 10
  case 'A' <= c && c <= 'F':
    return c - 'A' + 10
  }
  return 0
}
```

There's no automatic fall-through, but cases can be presented in comma-separated lists:

```go
func shouldEscape(c byte) bool {
  switch c {
  case ' ', '?', '&', '=', '#', '+', '%':
    return true
  }
  return false
}
```

`break` statements can be used to terminate a `switch` early. Sometimes, you might need to break out of a loop surrounding the switch statement, and this can be accomplished by putting a label on the loop and "breaking" to that label:

```go
Loop:
  for n := 0; n < len(src); n += size {
    switch {
    case src[n] < sizeOne:
      if validateOnly {
        break // breaks out of the switch
      }
      size = 1
      update(src[n])

    case src[n] < sizeTwo:
      if n+1 >= ln(src) {
        err = errShortInput
        break Loop  // breaks out of the loop
      }
      if validateOnly {
        break
      }
      size = 2
      update(src[n] + src[n+1]<<shift)

    }
  }
```

In Go, switch cases are evaluated in order, but you can put the `default` case first. That is, if `current == time.Monday` the message "It'll be Wednesday in two day's time!" will be displayed, because the default clause is not evaluated first.

```go
	current := time.Now().Weekday()  

	switch current {
	default:
		fmt.Println("Wednesday is more than two days away :(")
	case time.Wednesday:
		fmt.Println("Today is Wednesday!")
	case time.Tuesday:
		fmt.Println("Tomorrow is Wednesday!")
	case time.Monday:
		fmt.Println("It'll be Wednesday in two day's time!")
	}
```

### Type Switch

A switch can also be used to discover the dynamic type of an interface variable. 

Such a type switch uses the syntax of a type assertion with the keyword `type` inside the parentheses.

If the switch declares a variable in the expression, the variable will have the corresponding type in each clause.

It's considered idiomatic to reuse the name in such cases:

```go
var t interface{}
t = functionOfSomeType()
switch t := t.(type) {
default:
  fmt.Printf("unexpected type %T\n", t)
case bool:
  fmt.Printf("boolean %t\n", t)
case int:
  fmt.Printf("integer %t\n", t)
case *bool:
  fmt.Printf("pointer to boolean %t\n", *t)
case *int:
  fmt.Printf("pointer to int %t\n", *t)
}
```

Note again that the default clause is placed first.

## Functions

### Multiple Return Values

Go supports functions and methods that return multiple values. This feature is used to support some of the base C capabilities.

For example, in Go, the `Write` method is defined as:

```go
func (file *File) Write(b []byte) (n int, err error)
```

This allows you to inform the caller that some bytes were written but an error was found before completing the request (maybe you ran out of disk space while doing so).

This is a common style, and it is considered idiomatic.

Another useful scenario in which multiple return values are handy is illustrated in the following piece of code in which we have a function that returns the value at the given slice index, and the next position:

```go
func nextInt(b []byte, i int) (int, int) {
  for ; i < len(b) && !isDigit(b[i]); i++ {    
  }
  x := 0
  for ; i < len(b) && isDigit(b[i]); i++ {
    x = x*10 + int(b[i]) - '0'
  }
  return x, i
}
```

That function can then be used to scan the numbers in an input slice using:

```go
for i := 0; i < len(b); {
  x, i = nextInt(b, i)
  fmt.Println(x)
}
```

### Named Result Parameters

In Go, the return parameters can be given names and used as regular variables:

```go
func nextInt(b []byte, pos int) (value, nextPos int) {
  ...
}
```

When named, they're initialized to the zero values for their types, and if the function use a return statement with no arguments, the current values of the result parameters are used as the returned values.

It is considered idiomatic to do so, as they can make the code shorter and clearer &mdash; they are documentation.

### Defer

Go's defer statement schedules a function call (the *deferred* function) to be run immediately before the function executing the defer returns.

It's an unusual, but effective way to deal with situations such as resources that must be released regardless of which path a function takes to return.

The canonical example is closing a file:

```go
func Contents(filename string) (string, error) {
  f, err := os.Open(filename)
  if err != nil {
    return "", err
  }
  defer f.Close()   // f.Close will run right before exiting Contents

  var result []byte
  buf := make([]byte, 100)
  for {
    n, err := f.Read(buf[0:])
    result = append(result, buf[0:n]...)
    if err != nil {
      if err == io.EOF {
        break
      }
      return "", err  // f will be closed thanks to defer
    }
  }
  return string(result), nil // f will be closed thanks to defer
}
```

Using defer in this way has several advantages in terms of readability and maintainability:
+ it guarantees that the file will be closed, independently of how many branches you have in your code.
+ the `Close` is kept near the `Open` statement.

The arguments to the deferred function are evaluated when the `defer` statement is executed, not when the called to the deferred function is executed.

Deferred functions are executed in LIFO order.

As a result of the previous paragraphs the snippet:

```go
for i := 0; i < 5; i++ {
  defer fmt.Printf("%v ", i)
}
```

will print "4 3 2 1 0".

While peculiar, `defer` can be used to implement function-based (rather than block-based) solutions to certain problems such as error management or tracing.

| EXAMPLE: |
| :------- |
| See [05_tracing-defer](../../Part_3-misc-mini-projects/01_mini-projects/05_tracing-defer/README.md) for a simple example of how to use `defer` for tracing the execution flow. |

## Data

Go has two allocation primitives available as built-in functions: `new` and `make`.

They do different things and apply to different types.

### Allocation with new

`new` is a built-in function that allocates memory but does not initialize it, that is, it is not a constructor.

In technical terms, `new(T)` allocates zeroed storage for a new item of type `T` and returns its address, a value of type `*T` (it returns a pointer to a newly allocated zero value of type T).

Many data structures are designed in a way that exploit the zeroed memory to their benefit. For example you either do:

```go
p := new(SyncedBuffer)
var v SyncedBuffer
```

and start using `p` and `v` without further arrangement.

### Constructors and composite literals

Sometimes the zero value is not enough and an initializing constructor function is necessary:

```go
func NewFile(fd int, name string) *File {
  if fd < 0 {
    return nil
  }
  f := new(File)
  f.fd = fd
  f.name = name
  f.dirinfo = nil
  f.nepipe = 0
  return f
}
```

In Go, such a verbose initialization can be simplified using composite literal:

```go
func NewFile(fd int, name string) *File {
  if fd < 0 {
    return nil
  }
  f := File{fd, name, nil, 0}
  return &f
}
```

And that's one of the reasons you don't typically see that many `new(T)` in the wild.

Note also that unlike C, it's perfectly fine to return the address of a function's local variable. The storage associated will survive after the function returns. In fact, you can also take the address of a composite literal and return it as it allocates a fresh instance each time it is evaluated:

```go
func NewFile(fd int, name string) *File {
  if fd < 0 {
    return nil
  }
  return &File{fd, name, nil, 0}
}
```



The fields of a composite literal are laid out in order and must all be present as seen in `File{fd, name, nil, 0}`.

However, you can use labels to put the fields in any order, and in that case, you can also benefit from the zero values of the corresponding fields:

```go
func NewFile(fd int, name string) *File {
  if fd < 0 {
    return nil
  }
  return &File{fd: fd, name: name}
}
```

If the composite literal contains no fields you can either do `new(File)` or `&File{}` and you'll obtain the same results.

Composite literals can also be created for arrays, slices, and maps, with the field labels being indices or map keys as appropriate.

```go
const (
	Enone int = iota
	Eio
	Einval
)

func main() {
  // array: [no error, eio, invalid argument]
	a := [...]string {Enone: "no error", Eio: "eio", Einval: "invalid argument" }

  // slice: [no error, eio, invalid argument]
	s := []string {Enone: "no error", Eio: "eio", Einval: "invalid argument" }

  // map[0: no erro, 1: eio, 2: invalid argument]
	m := map[int]string{Enone: "no error", Eio: "eio", Einval: "invalid argument" }
}
```

### Allocation with make

The built-in function `make(T, args)` serves a purpose different from `new(T)`.

It creates slices, maps, and channels only, and it returns an initialized (not zeroed) value of type `T` (not `*T`).

The reason for the distinction is that these three types represent under the covers references to data structures that must be initialized before use.

A slice for example is a three-item descriptor, containing a pointer to the data inside an array, the length, and the capacity. Until those items are initialized the slice is `nil`, not a zeroed memory area.

As a result, a call to `make` initializes the internal data structure and prepares the value for its use:

```go
// allocates an array of 100 ints and create a slice data structure
// with length = 10 and capacity of 100 pointing to the first 10 elements of the array
make([]int, 10, 100)
```

In contrast, `new([]int)` returns a pointer to a nil slice value. Statements such as:

```go
var p *[]int = new([]int)
```

are rarely useful.

| NOTE: |
| :---- |
| Remember that `make` applies only to maps, slices, and channels, and that does not return a pointer.<br>If you need the pointer allocate with `new` or better yet, take the address of the variable explicitly |

### Arrays

Arrays are useful when planning the detailed layout of memory and can be used to avoid unnecessary allocation.

In practice, they're used primarily as the building blocks for slices.

You should know that:
+ unlike in C, arrays are values. Assigning one array to another copies all the elements.
+ If you pass an array to a function, you will receive a copy of the array, not a pointer to it.
+ The size of an array is part of its type. The types `[10]int` and `[20]int` are different.

While you can pass an array by address to a function to get a C-like behavior:

```go
func Sum(a *[3]float64) (sum float64) {
  for _, v := range *a {
    sum += v
  }
  return
}

array := [...]float64{3.0, 5.5, 9.6}
x := Sum(&array)
```

That is not considered idiomatic, and slices should be preferred instead.

### Slices

Slices wrap arrays to give more general, powerful, and convenient interface to sequences of data. Except for items with explicit dimensions such as transformation matrices, most array programming in Go is done with slices rather than arrays.

Slices hold references to an underlying array, and if you assign one slice to another, both refer to the same array.

If a function takes a slice argument, changes made to the elements of the slice will be visible to the caller, analogous to passing a pointer to the underlying array.

As a result, the signature of a `Read` method can look like:

```go
func (f *File) Read(buf []byte) (n int, err error) {
  ...
}
```

That is, you don't need to pass a pointer to buf.

To read into the first 32 bytes of a larger buffer you can do:

```go
n, err := f.Read(buf[0:32])
```


The length of a slice may be changed as long as it still fits with the limits of the underlying array.

The capacity of the slice, accesible by `cap`, reports the maximum length the slice may assume.

The following function appends data to a slice. If the data exceeds the capacity, the slice is reallocated, and the resulting slice is returned. Note that `len` and `cap` are applied to the `nil` slice (as you can rightfully do), and they return 0.

```go
func Append(slice, data []byte) []byte {
  l := len(slice)
  if l + len(data) > cap(slice) {
    // Allocated double what's needed for future growth
    newSlice := make([]byte, (l+len(data))* 2)
    copy(newSlice, slice) // copy is predeclared and works for any slice type
    slice = newSlice
  }
  slice = slice[0:l + len(data)] // enlarge to l + len(data)
  copy(slice[l:], data)          // copy the new data
  return slice                   // return the modified slice
}
```

Note that we must return the slice because `Append` can modify the elements of `slice`, but the slice itself (the run-time data structure holding the pointer, length, and capacity) is passed by value, so it will revert back to the original underlying backing array, length, and capacity if we returned what's passed as argument.

### Two-dimensional slices

To create 2D arrays or slices you need to define an array of arrays or slice of slices:

```go
type Transform[3][3]float64 // 3x3 array
type LinesOfText [][]byte   // a slice of byte slices
```

As slices are variable-length, it is possible to have each inner slice be a different length:

```go
text := LinesOfText{
  []byte("Now is the time"),
  []byte("for all good gophers"),
  []byte("to bring some fun to the party"),
}
```

Sometimes it is necessary to allocate a 2D slice (for example, when processing images as a matrix of pixels).
This can be done by either allocating each slice independently, or alternatively to allocate a single array and point the individual slices to it.

An example of the first method:

```go
picture := make([][]uint8, YSize)
for i := range picture {
  picture[i] = make([]uint8, XSize)
}
```

An example of the second method:

```go
picture := make([][]uint8, YSize)

// Allocate one large slice to hold all the pixels
pixels := make([]uint8, XSize*YSize)

// Loop over the rows, slicing each row from the front of the remaining pixels slice
for i := range picture {
  picture[i], pixels = pixels[:XSize], pixels[XSize:]
}
```

### Maps

In Go, maps are a convenient built-in data structure that associate values of one type (the key) with values of another type (the element or value).

The key can be of any type for which the equality operator is defined (integers, floating point, complex numbers, strings, pointers, interfaces, structs, and arrays). 

| NOTE: |
| :---- |
| Slices cannot be used as map keys because equality is not defined on them. |

Maps hold references to an underlying data structure. As a result, if you pass a map to a function that changes the contents of the map, the changes will be visible in the caller.

Maps can be built using the composite literal syntax:

```go
var timeZone = map[string]int{
  "UTC": 0*60*60,
  "EST": -5*60*60,
  "CST": -6*60*60,
}
```

Assigning and fetching maps looks syntactically just like doing the same for arrays and slices:

```go
offset := timeZone["EST"]
```

An attempts to fetch a map value with a key that is not present will return the zero value for the type of the entries in the map:

```go
attended := map[string]bool{
  "Harry": false,
  "Mark":  true,
  "John": true,
}

if attended[person] {
  fmt.Println(person, "was at the meeting")
}
```

If you need to distinguish a missing entry from the zero value you can do:

```go
var seconds int
var ok bool
seconds, ok = timeZone[tz]
```

| NOTE: |
| :---- |
| This idiom is called the "*comma ok*" idiom. |

```go
func offset(tz string) int {
  if seconds, ok := timeZone[tz]; ok {
    return seconds
  }
  log.Println("unknown time zone", tz)
  return 0
}
```

To delete a map entry, you can use the delete built-in function:

```go
delete(timeZone, "PDT")
```

### Printing

Formatted printing in Go is similar to C but richer and more general.

The functions live in the `"fmt"` package and have capitalized names such as `fmt.Printf`, `fmt.Fprintf`, and `fmt.Sprintf`.

For each of `Printf`, `Fprintf`, and `Sprintf`, there is an instance of `Print` and `Println` so that you don't have to provide a format string.

The `Println` version inserts a blank between arguments and append a newline to the output.

The `Print` version add blanks only if the operand on neither side is a string.

The formatted print functions `fmt.Fprintf` and friends take any object implementing the `io.Writer` interface as the first argument. The variables, `os.Stdout` and `os.Stderr` are familiar instances.

While you can use `"%d"` for numbers, `"%x"` for hex, etc. if you want the default conversion you can use the *catchall* format `"%v"`. This even works for arrays, slices, structs, and maps.

When printing a struct, the modified format `"%+v"` prints also the structure with their names, and the `"%#v"` prints the value in full Go syntax.

```go
type T struct {
  a int
  b float64
  c string
}

t := &T{7, -2.35, "abc\tdef"}
fmt.Printf("%v", t)  // &{7 -2.35 abc    def}
fmt.Printf("%+v", t) // &{a:7 b:-2.35 c:abc    def}
fmt.Printf("%#v", t) // &{a:7, b:-2.35, c:"abc\tdef"}
```

The quoted format is available through `"%q"` when applied to a `string` or `[]byte`. The alternate format `"%#q"` will use backquotes if possible.

The format `"%x"` produces a long hex string, and `"% x"` puts a space between bytes.

The format `%T` prints the type of a value.

To control the default format for a for a custom type, you just need to define a method satisfying the stringer interface:

```go
func (t *T) String() string {
  return fmt.Sprintf("...")
}
```

If you need to print values of type `T` as well as pointers to `T` (that is, `*T`), the receiver for String must be of value type.

You must not construct a `String` method by calling `Sprintf` in a way that will recur ito your `String` method indefinitely.

This can happen if the `Sprintf` call attempts to print the receiver directly as a string, which in turn will invoke the method again:

```go
type MyString string

func (m MyString) String() string {
  return fmt.Sprintf("MyString=%s", m)  // will recur forever
}
```

A simple way to fix this is to make sure we're not invoking our custom `String` method:

```go
func (m MyString) String() string {
  return fmt.Sprintf("MyString=%s", string(m))
}
```

Another printing technique is to pass a a print routine's arguments directly to another similar routine.

The signature of such print routines are typically:

```go
func Printf(format string, v ...interface{}) (n int, err error)
```

Thus, you can do:

```go
func Println(v ...interface{}) {
  std.Output(2, fmt.Sprintln(...v))
}
```

| NOTE: |
| :---- |
| The syntax `...v` tells the compiler to treat `v` as a list of arguments, instead of as a single array/slice argument. |

### Append

The signature of append is as follows:

```go
func append(slice []T, elements ...T) []T
```

where `T` is a placeholder for any given type.

This function appends elements to the end of the slice and return the result.

```go
x := int[]{1, 2, 3}
x = append(x, 4, 5, 6) // x == int[]{1, 2, 3, 4, 5, 6}
```

To append a slice to a given slice you need to use the `...` in the invocation:

```go
x := int[]{1, 2, 3}
y := int[]{4, 5, 6}
x = append(x, y...) // x == int[]{1, 2, 3, 4, 5, 6}
```


## Initialization

Initialization in Go is more powerful than in other languages. Complex structures can be built during initialization, and the ordering issues are handled in a correct fashion.

### Constants

Constants in Go are created at compile time and can only be numbers, runes, strings or booleans.

The expressions that define them must be constant expressions evaluatable by the compiler. For that reason, `math.Sin(math.Pi/4)` cannot be a constant because it requires a runtime call to `math.Sin` that cannot happen at compile time.

Enumerated constants are created using the *iota* enumerator:

```go
type ByteSize float64

const (
  _ = iota  // ignore first value by assigning to blank identifier
  KB ByteSize = 1 << (10 * iota)
  MB
  GB
  TB
  PB
  EB
  ZB
  YB
)
```

The ability to attach a method to any type makes it very convenient in this case:

```go
func (b ByteSize) String() string {
  switch {
  case b >= YB:
    return fmt.Sprintf("%.2fYB", b / YB)
  case b >= ZB:
    return fmt.Sprintf("%.2fZB", b / ZB)    
  case b >= EB:
    return fmt.Sprintf("%.2fEB", b / EB)
  case b >= PB:
    return fmt.Sprintf("%.2fPB", b / PB)
  case b >= TB:
    return fmt.Sprintf("%.2fTB", b / TB)
  case b >= GB:
    return fmt.Sprintf("%.2fGB", b / GB)    
  case b >= MB:
    return fmt.Sprintf("%.2fMB", b / MB)
  case b >= KB:
    return fmt.Sprintf("%.2fKB", b / KB)    
  }
  return fmt.Sprintf("%.2fB", b)
}
```

### Variables

Variables can be initialized like constants but the initializer can be a general expression that can be computed at run time.

```go
var (
  home = os.Getenv("HOME")
  user = os.Getenv("USER")  
)
```

### The init function

Go allows for each source file to define a `init` function to set up whatever state is required.

Besides initialization that cannot be expressed as declarations, a common use for this `init` functions is to verify or repair correctness before real execution begins:

```go
func init() {
  if user == "" {
    log.Fatal("$USER not set")
  }

  if home == "" {
    home = "/home/" + user
  }
}
```

## Methods

### Pointers vs. Values

Methods in Go can be define for any named type (except for a pointer or an interface). That is, the receiver does not have to be a struct.

For example, we can define our custom `Append` function on slices:

```go
type ByteSlice []byte

func (slice ByteSlice) Append(data []byte) byte[] {
  ...
}
```

Note that this still requires the function to return a `byte[]`.

This can be solved by redefining the method to take a pointer to a `ByteSlice` so that the method can overwrite the caller's slice:

```go
func (p *ByteSlice) Append(data []byte) {
  slice := *p
  // Run the Append logic
  *p = slice
}
```

This can be further improved so that it looks like a standard `Write` method, and therefore satisfies the `io.Writer` interface:

```go
func (p *ByteSlice) Write(data []byte) (n int, err error) {
  slice := *p
  // Run the Append logic
  *p = slice
  return len(data), nil
}
```

After doing do, we can do:

```go
var b ByteSlice
fmt.Fprintf(&b, "This hour has %v days\n", 7)
```

and get the result of the formatting *loaded* into the `ByteSlice`.

The rule about pointers vs. values for receivers is that value methods can be invoked on pointers and values, but pointer methods can only be invoked on pointers.

This rule arises because pointer methods can modify the receiver; invoking them on a value would cause the method to receive a copy of the value, so that any modifications would be discarded.

The language therefore disallows this mistake. There is an exception though, when the value is addressable, the language takes care of the common case of invoking a pointer method on a value by inserting the address operator automatically.

## Interfaces and other types

### Interfaces

Interfaces in Go provide a way to specify the behavior of an object: 
> *if something can do this, it can be used here*.

Interfaces with only one (or two) methods a common in Go, and are usually given a name derived from the method such as `io.Writer` for the interface that implements `Write`, `Stringer` for the interface that implements `String()`, etc.

A type can implement multiple interfaces. For example, a collection can be sorted by the routines in package `sort` if it implements the `sort.Interface` which contains `Len()`, `Less(i, j int) bool`, and `Swap(i, j int)`. Additionally, the type can implement a custom formatter (implement the `String()` method).

```go
package main

import (
	"fmt"
	"sort"
)

type sequence []int


// sequence implement the sort.Interface
func (seq sequence) Len() int {
	return len(seq)
}

func (seq sequence) Swap(i, j int) {
	seq[i], seq[j] = seq[j], seq[i]
}

func (seq sequence) Less(i, j int) bool {
	return seq[i] < seq[j]
}

// Copy returns a copy of the sequence
func (seq sequence) Copy() sequence {
	res := make(sequence, 0, cap(seq))
	return append(res, seq...)
}

// sequence also implements the stringer interface
func (seq sequence) String() string {
	aux := seq.Copy()
	sort.Sort(aux)

	str := "["
	for i, elem := range aux {
		if i > 0 {
			str += " "
		}
		str += fmt.Sprint(elem)
	}
	return str + "]"
}


func main() {
	seq := sequence{4, 1, 2, 5, 3}
	fmt.Println(seq)  // [1 2 3 4 5]
}
```

| EXAMPLE: |
| :------- |
| See [Sequence](../../Part_3-misc-mini-projects/01_mini-projects/13_interfaces-sequence/) for a runnable example. |

### Conversions

The `String` method of `Sequence` recreates what the `Sprint` method already does for slices. As a result, you can do:

```go
// sequence also implements the stringer interface
func (seq sequence) String() string {
	aux := seq.Copy()
	sort.Sort(aux)
  return fmt.Sprint([]int(aux))
}
```

It is safe to call `Sprintf` ever if the two types (`Sequence` and `[]int`) are the same if we ignore the type name.

The conversion doesn't create a new value, it just temporarily acts as through the existing value has a new type. That is, it is similar to converting and integer to a float.

It's an idiom in Go programs to convert the type of an expression to access a different set of methods:

```go
type Sequence []int

// sequence also implements the stringer interface
func (seq sequence) String() string {
	aux := seq.Copy()
  sort.IntSlice(s).Sort()
  return fmt.Sprint([]int(s))
}
```

In the snippet below, we convert `Sequence` to an `IntSlice` and as a result, we no longer need to implement the `sort.Interface`.

### Interface conversions and type assertions

Type switches are a form of conversion: they take an interface and, for each case in the switch, convert it to the type of that case (in a sense).

```go
type Stringer interface {
  String() string
}

var value interface{}
switch str := value.(type) {
case string:
  return str
case Stringer:
  return str.String()
}
```

A type assertion takes an interface value and extracts from it a value of the specified explicit type:

```go
// Generic
t := value.(typeName)

// Example using string no guards (will panic if value does not contain a string)
str := value.(string)

// Example using string and guard
str, ok := value.(string)
if ok {
  fmt.Printf("string value is %q\n", str)
} else {
  fmt.Printf("value is not a string")
}
```

The type switch found at the beginning of the section can be rewritten as:

```go
if str, ok := value.(string); ok {
  return str
} else if str, ok := value.(Stringer); ok {
  return str.String()
}
```

Either way, you can check whether some specific value is either an specific type (`string`) or implements a certain interface `Stringer`.

### Generality

If a type exists only to implement an interface and will never export methods there is no need to export the type itself.

Actually, exporting just the interface makes it clearer that the value has no interesting behavior beyond what is described in the interface.

This happens in the hash libraries. Both `crc32.newIEEE` and `adler32` return the interface type `hash.Hash32`. Therefore, you can substitute the `CRC-32` algorithm for `Adler-32` by changing only the constructor call and nothing more.

### Interfaces and methods

Since almost anything can have methods attached, almost anything can satisfy an interface.

For example, the `http` package defines the `Handler` interface. Any object that implements `Handler` can serve HTTP requests:

```go
type Handler interface {
  ServeHTTP(ResponseWriter, *Request)
}
```

`ResponseWriter` is an interface that provides access to the methods needed to return the response to the client. Those methods include the standard `Write` method, so an `http.ResponseWriter` can be used wherever an `io.Writer` can be used. `Request` is a struct containing a parsed representation of the request received from the client.

You can make a struct, a plain int, a channel, or even a regular function (not a method) implement that interface.

| EXAMPLE: |
| :------- |
| See [15_http-http-counter](../../Part_3-misc-mini-projects/01_mini-projects/15_http-http-counter/README.md) for a runnable example. |

## The blank identifier

The blank identifier `_` can be assigned or declared with any value of any type, with the value discarded harmlessly.

### The blank identifier in multiple assignment

If an assignment requires multiple values on the left side, but one of the values will not be used by the program, a blank identifier on the left-hand side of the assignment avoids the need to create a dummy variable and makes it easy to understand that the value is to be discarded.

```go
if _, err := os.Stat(path); os.IsNotExist(err) {
  fmt.Printf("%s does not exist\n", path)
}
```

### Unused imports and variables

In Go, it is considered an error to import a package or to declare a variable without using it.

As a workaround when doing development, you can use the blank identifier to silence those complains:

```go
import (
  "fmt"   // unused
  "io"    // unused
  "log"
  "os"
)

var _ = fmt.Printf  // silence fmt package unused
var _ = io.Reader   // silence io package unused

...

fd, err := os.Open("data.csv")  // fd is not used yet
if err != nil { ... }

_ = fd // silence fd is not used
```

### Import for side effect

Sometimes it is useful to import a package for its side effects, without any explicit use.

For example, you might want the package `init` function to be called (as happens with `net/http/pprof` which registers some handlers).

The best way to do so is to use the blank identifier:

```go
import _ "net/http/pprof"
```

### Interface checks

A type in Go does not need to declare explicitly that it implements an interface. Instead, a type implements the interface just by implementing the interface methods.

In practice, most interface conversions are static (e.g. passing an `*os.File` to a function expecting an `io.Reader` won't compile unless the pointer to `os.File` implements the `io.Reader` interface).

However, some interface checks do happen at runtime (e.g., the `encoding/json` package defines a `Marshaler` interface). When the JSON encoder receives a value that implements that interface, the encoder invokes the value's marshaling method to convert it to JSON.

That type of runtime interface check is done with a type assertion:

```go
m, ok := val.(json.Marshaler)
```

## Embedding

Go does not provide the typical, type-drive notion of subclassing, but it does have the ability to to borrow pieces of an implementation by embedding types within a struct or interface.

```go
type Reader interface {
  Read(p []byte) (n int, err error)
}

type Writer interface {
  Write(p []byte) (n int, err error)
}
```

There is a `ReadWriter` interface in the `io` pacckage defined as:

```go
type ReadWriter interface {
  Reader
  Writer
}
```

And the same basic idea applies to structs:

```go
type ReadWriter struct {
  reader *Reader
  writer *Writer
}

When using the approach above, you will need to explicitly use the `reader` when forwarding to the corresponding method.
...
func (rw *ReadWriter) Read(p []byte) (n int, err error) {
  return rw.reader.Read(p)
}
```

However, Go also allows you to define the struct as:

```go
type ReadWriter struct {
  *Reader
  *Writer
}
```

which automatically surfaces the corresponding methods on the struct itself.

For example, you can do:

```go
type Job struct {
  Command string
  *log.Logger
}
```

which will automatically surface in the corresponding job values the `log.Logger` methods:

```go
job.Printf("Starting job for command %q\n", job.Command)
```

And the `Logger` field is a regular field of the `Job` struct, so it can be initialized in the usual way:

```go
// composite literal
job := Job{
  Command: "cp README.md README_OLD.md",
  Logger:  log.Default(), // alternatively, you can customize doing `log.New(os.Stderr, "Job": ", log.Ldate)`
}

// constructor
func NewJob(command string, logger *log.Logger) *Job {
  return &Job{command, logger}
}

job2 := NewJob("cp README.md README_OLD.md", log.Default())
```

If we need to refer to an embedded field directly, the type name of the field, ignoring the package qualifier, serves as a field name (e.g., `job.Logger` to refer to the `*log.Logger`).


That will be usefult if we want to refine the methods on the logger:

```go
func (job *Job) Printf(format string, args ...interface{}) {
  job.Logger.Printf("%q: %s", job.Command, fmt.Sprintf(format, args))
}
```

In this case, any name conflict are resolved using two simple rules:

1. A field or method X hides another X in a more deeply nested part of the type (e.g. `job.Printf` will hide `log.Logger.Printf`).

2. If the same name appears at the same nesting level, it is usually  an error (e.g., the `Job` featuring a `Logger` field). However, if the duplicate name is never mentioned in the program outside of the type definition it will be allowed (this provides some protection against changes made to types embedded from outside).

## Concurrency

### Share by communicating

Go encourages a different approach from traditional concurrent programming (which is based on providing constructs to implement correct access to shared variables). Instead, in Go shared values are passed around on channels, and never actively shared by separate threads of execution. Only one goroutine has access to the value at any given time, so race condition on data cannot occur by design.

> Do not communicate by sharing memory; instead, share memory by communicating.

This high-level approach consisting in using channels to control access makes it easier to write clear, correct programs. That said, Go also provides the low-level constructs (e.g. mutex) to let you synchronize access to shared variables.

| NOTE: |
| :---- |
| Go's approach to concurrency originates in Hoare's Communcating Sequential Processes (CSP), but it can also be seen as a type-safe generalization of Unix pipes. |

### Goroutines

A goroutine has a simple model: it is a function executing concurrently with other goroutines in the same address space. It is lightweight, costing little more than the allocation of stack space.

Goroutines are multiplexed onto multiple OS threads, so if one should block, such as while waiting for I/O, others continue to run. Their design hides many of the complexities of thread creation and management.

To use them, you just need to prefix a function or method call with the `go` keyword to run the call in a new goroutine. When the call completes, the goroutine exits silently.

| NOTE: |
| :---- |
| You can think of a goroutine as the equivalent of running a Unix command in the background using `&`. |

For example, you can do:

```go
go list.Sort()  // run list.Sort concurrently, don't wait for it to complete
```

A function literal is also common:

```go
func Announce(message string, delay time.Duration) {
  go func() {
    time.Sleep(delay)
    fmt.Println(message)
  }() // function needs to be invoked
}
```

### Channels

Channels are allocated with `make` and the resulting value acts as a reference to the underlying data structure (as happens with maps and slices).

When allocating channels, you can provide an optional integer that sets the buffer size for the channel, with the default being zero (unbuffered or synchronous channel).

```go
ci := make(chan int)    // unbuffered channel of integers
cj := make(chan int, 0) // unbuffered channel of integers
cs := make(chan *os.File, 100) // buffered channel of pointers to Files
```

Unbuffered channels combine communication with synchronization, guaranteeing that two pieces of logic implemented as goroutines are in a known state.

This can be used to implement nice *idioms* such as running a piece of logic asynchronously while your main program is doing something else.

```go
c := make(chan int)   // Allocate unbuffered channel for sync purposes
go func() {
  list.Sort() // this may take a while
  c <- 1      // Send a "done" signal
}()
doSomethingWhileSorting()
<-c // blocks until signal received
doSomethingOnceSortingDone()
```

A buffered channel can be used like a semaphore, for instance to limit throughput.

```go
var sem = make(chan int, MaxOutstanding)

func handle(r *Request) {
  sem <- 1    // Wait for active queue to drain
  process(r)
  <-          // Done: enable next request to run
}

func Serve(queue chan *Request) {
  for {
    req := <-queue
    go handle(req)    // Don't wait for handle to finish
  }
}
```

Once `MaxOutstanding` handlers are executing `process`, any additional request will block trying to send into the filled channel, until one of the existing handlers finishes and receives from the buffer.

This has problem though, because although only `MaxOutstanding` requests can be processed at any given time, a goroutine is created for each and every of them that is received.

| EXAMPLE: |
| :------- |
| See [Concurrency: Rate limiter](../../Part_3-misc-mini-projects/01_mini-projects/17_concurrency-rate-limiter/rate-limiter-v0/) for a runnable example. |

It would seem that in order to fix the problem, we would just need to change `Serve` to gate the creation of the goroutines:

```go
// we get rid of handle, and use Serve as both the TaskServer and handler
func Serve(queue chan *Request) {
  for req := range queue {
    go func() {
      process(req)  // Buggy: keep reading for explanation
      <-sem
    }()
  }
}
```

The buf is that in Go, the loop variable is reused for each iteration, so the `req` variable will be shared across all goroutines and therefore potentially creating problems.

To prevent concurrency problems, we need to ensure that we have a unique variable in each goroutine. We can do that by either passing the value of `req` as an argument to the closure:

```go
func Serve(queue chan *Request) {
  for req := range queue {
    go func(req *Request) {
      process(req)  // Buggy: keep reading for explanation
      <-sem
    }(req)
  }
}
```

or alternatively, by creating a new variable with the same name:

```go
func Serve(queue chan *Request) {
  for req := range queue {
    req := req      // Force the creation of a new instance in each iteration
    go func() {
      process(req)  // No longer buggy
      <-sem
    }()
  }
}
```

While it may seem odd to write:

```go
req := req
```

it's legal and idiomatic to do so in Go. You get a fresh version of the variable with the same name that is deliberately shadowing the loop variable locally, so that you get a unique variable for each loop iteration and therefore goroutine.

### Channels of channels

One of the most important properties of Go is that a channel is a first-class value that can be allocated and passed around like any other.

A common use for this scenario is to implement safe, parallel demultiplexing.

For example, we could create an schematic definition of the `Request` structure defined in the previous exercise that includes a channel on which to reply, so that each client can provide its own path for the answer.

```go
type Request struct {
  args       []int
  f          func([]int) int
  resultChan chan int
}
```

That way, we could create a request like:

```go
func sum(a []int) (s int) {
  for _, v := range a {
    s += v
  }
  return
}

request := &Request{[]int{3, 4, 5}, sum, make(chan int)}

// Send request
clientRequests <- request

// Wait for response
fmt.Printf("answer: %d\n", <-request.resultChan)
```

And this only requires a small change on the handler function to execute the function passed on the request, and provide the result to the channel.

```go
func handle(queue chan *Request) {
  for req := range queue {
    req.resultChan <- req.f(req.args)
  }
}
```

While this is a simplification, this illustrates how easy it is to build a rate-limited, parallel, non-blocking RPC system, without having to use any of the classical concurrency constructs like mutexes.

### Parallelization

In Go, you can also parallelize a calculation across multiple CPU cores.

If a calculation can be broken down into separate pieces than can execute independently, it can be parallelized, and you can use a channel to signal when each piece completes.

Consider the following example in which we have an expensive operation to be executed on a vector of items, and that the value of the operation on each item is independent:

```go
type Vector []float64

// Apply the operation to v[i], v[i+1], ... v[n-1]
func (v Vector) DoSome(i, n int, u vector, c chan int) {
  for ; i < n; i++ {
    v[i] += u.Op(v[i])
  }
  c <-1 // signal "done"
}
```

We can launch the pieces independently in a loop, one per CPU. They can complete in any order, we just count the completion signals by draining the channel until all individual operations are done:

```go
const numCPU = 4 // num of CPU cores

func (v Vector) DoAll(u vector) {
  c := make(chan int, numCPU) // buffer with the number of messages to be received
  for i := 0; i < numCPU; i++ {
    fo v.DoSome(i*len(v)/numCPU, (i+1)*len(v)/numCPU, u, c)
  }

  // Drain the channel
  for i := 0; i < numCPU; i++ {
    <-c // wait for one task to complete
  }
  // All done
}
```

You can also interrogate the runtime, to get the number of available CPUs and size the parallel processing accordingly:

```go
var numCPU = runtime.NumCPU()
```

There is also a function `runtime.GOMAXPROCS` which reports or sets the user-specified number of cores that a Go program can have running simultaneosly.

It defaults to the value of `runtime.NumCPU` but can be overridden by setting an environment variable named `GOMAXPROCS` or by calling the  function with a positive number.

Calling it with zero as argument queries the current value.

As a result, we can do:

```go
var numCPU = runtime.GOMAXPROCS(0)
```

| NOTE: |
| :---- |
| It's important to distinguish the ideas of concurrency (being able to independently execute parts of a program) and parallelism (executing calculations in parallel for efficiency on multiple CPUs). Go is a concurrent language, not a parallel one, although some parallelization problems can be solved thanks to Go's concurrent features. |

## Errors

Library routines must often return some sort of error indication to the caller.

Go's multivalue return makes it easy to return a detailed error description alongside the normal return value.

It's considered good style to use this feature to provide detailed error information. For example, `os.Open` doesn't just return a `nil` pointer to the File on failure, it also returns an error value that describes what went wrong.

By convention, errors have a type `error` defined as:

```go
type error interface {
  Error() string
}
```

A library writer is free to implement this interface with a richer model. For example:

```go
type PathError struct {
  Op    string  // open, unlink, etc.
  Path  string  // associated path to file
  Err   error   // the actual error returned by the system call
}

func (e *PathError) Error() string {
  return e.Op + " " + e.Path + ": " + e.Err.Error()
}
```

When feasible, error strings should identify their origin as in: `image: unknown format`.

When interested about the precise error details, a type switch or type assertion can be used:

```go
for try := 0; try < 2; try++ {
  file, err = os.Create(filename)
  if err == nil {
    return
  }
  if e, ok := err.(*os.PathError); ok && e.Err == syscall.ENOSPC {
    deleteTempFiles()
    continue
  }
  return
}
```

| NOTE: |
| :---- |
| You can also use `errors.IsError()` to identify the type. You can also define your own set of sentinel errors similarly to `syscall.ENOSPAC` or `io.EOF`. You can review the corresponding section and an example in [24_errors-sentinel-errors](../../Part_1-basic-intro/04_intro_methods_interfaces_type-assertions/24_errors-sentinel-errors/). |

### Panic

When your program finds an unrecoverable error (i.e., the program can't continue executing), the `panic` built-in function creates a run-time error that will stop the program.

The function takes a single argument of an arbitrary type (typically a string) that will be printed as the program dies.

For example:

```go
func SquareRoot(x float64) float64 {
  ...
  for i := 0; i < 1e6; i++ {
    ...
    if veryClose(...) 
    return result
  }
  // if we get here, the alg hasn't converged in 1 millio iterations
  panic(fmt.Sprintf("SquareRoot(%g) did not converge", x))
}
```

| NOTE: |
| :---- |
| It's always better to let the things continue rather than taking down the whole program using `panic`. |

### Recover

When `panic` is called (either explicitly by your program, or implicitly because your program do such thing as indexing an array out of bounds), it immediately stops exectution of the current function and begins unwinding the stack of the goroutine, running any deferred functions along the way.

If that unwinding reaches the top of the goroutine's stack, the program dies. However, it is possible to use the built-in function `recover` to regain control of the `goroutine` and resume normal execution.

A call to recover stops the unwinding and returns the argument passed to `panic`. 

| NOTE: |
| :---- |
| As the only code that runs while unwinding is inside deferred functions, `recover` is only useful inside deferred functions. |

Consider the following snippet in which you implement recover to safely terminate a task handler that has panicked, but you keep doing the work in the other ones:

```go
func server(workChan <- chan *Work) {
  for work := range workChan {
    go safelyDo(work)
  }
}

func safelyDo(work *Work) {
  defer func() {
    if err := recover(); err != nil {
      log.Println("work failed:", err)
    }
  }()
  do(work)
}
```

If `do(work)` panics, the result will be logged and the goroutine will exit cleanly without disturbing the others &mdash; no need to do anything else in the deferred closure; recover will handle the condition completely.

Note also that because `recover` returns `nil`  unless called directly from a deferred function, deferred code can call library routines that use `panic` and `recover` without failing.

The *recovery pattern* illustrated above can be applied to multiple situations in which we need to simplify error handling.

In the following snippet, `panic` is used to report parsing errors in a regexp package:

```go
type Error string

func (e Error) Error() string {
  return string(e)
}

func (regexp *Regexp) error(err string) {
  panic(Error(err))
}

func Compile(str string) (regexp *Regexp, err error) {
  regexp = new(Regexp)

  // doParse might panic if a parsing error is found
  defer func() {
    if e := recover(); e != nil {
      regexp = nil      // Clear return value
      err = e.(Error)   // Cause an "implicit" panic if not a parse error
    }()
    return regexp.doParse(str), nil
  }
}
```

Note that if the type assertion `err = e.(Error)` fails, it'll cause a run-time error and won't recover. Otherwise, the recovery block will set the returned value to nil, and return the parsing error without exposing the panic to the client library consumers.

| NOTE: |
| :---- |
| Not exposing internal *panics* to library clients is a good rule to follow. |

The `error` method is a convenience method to report parsing errors, so that you can do:

```go
if pos == 0 {
  re.error("'*' illegal at start of expression")
}
```

An cause a panic as a result.

## A web server

> This example illustreates a complete Go program.

Google provides a service at `chart.apis.google.com` that does automatic formatting of data into charts and graphs.

It's hard to use interactively, because you need to put the data into the URL as a query. In the example, we are providing a nicer interface to one form of data: given a short piece of text, it calls on the chart server to produce a QR code. The image can then be grabbed by your phone, and interpreted as a URL saving you from having to type the URL into the `chart.apis.google.com` site.

## ToDo:
+ create an example with the panic/recovery pattern.